# Introduction to Data Science
## Homework 2

Student Name: Chutang Luo 

Student Netid: cl5293


### Part 1: Case study (5 Points)
- Read [this article](http://www.nytimes.com/2012/02/19/magazine/shopping-habits.html) in the New York Times.
- Use what we've learned in class and from the book to describe how one could set Target's problem up as a predictive modeling problem, such that they could have gotten the results that they did.  Formulate your solution as a proposed plan using our data science terminology.  Include all the aspects of the formulation that you see as relevant to solving the problem.  Be precise but concise.

Target needs to identify a customer's status as quickly as possible, so that it can deliver certain advertisements to the customer more precisely, which will guide the customer to buy something they need even though he/she has not realized it and will create more profit for the company. 

To identify a customer's, Target needs to determine what statistical model will be suitable to use and coleect the corresponding data. For example, to identify a customer is pregnant or not, logistic regression model can be used because the customer will be catogorized into two types: the customer is pregnant and the customer is not pregnant. If we need to categorize customers into several types, decision tree model can be used. 

Before building the model, the company needs to carefully collected the data and analyze what features will have influence on the target variable. To predict a customer is pregnant or not, then the data related to whether the customer buy something about babies should be collected. Different target variables needs different kinds of data. Also, historical data should be collected and used to train and test the model. Knowing that the customer was pregnant, we need to find what buying behaviour this customer had conducted. After the model is built and improved, the company can finally put it into use, predicting the status of the customer and correspondingly deliver a more precise advertisement.


### Part 2: Exploring data in the command line (4 Points - 1 each)
For this part we will be using the data file located in `"advertising_events.csv"`. This file consists of records that pertain to some online advertising events on a given day. There are 4 comma separated columns in this order: `userid`, `timestamp`, `domain`, and `action`. These fields are of type `int`, `int`, `string`, and `int` respectively. Answer the following questions using Linux/Unix bash commands. All questions can be answered in one line (sometimes, with pipes)! Some questions will have many possible solutions. Don't forget that in IPython notebooks you must prefix all bash commands with an exclamation point, i.e. `"!command arguments"`.

[Hints: You can experiment with whatever you want in the notebook and then delete things to construct your answer later.  You can also use ssh to use the actual bash shell in your terminal and then just paste your answers here. Recall that once you enter the "!" then filename completion should work. Also, these are standard data exploration commands that are quick and easy to use in a terminal or in the notebook. We don't cover command line operations formally in this class, but these are worth learning (and thus are part of the HW). Be resourceful. Use whatever online cheat sheets or Stackoverflow to answer the question.]

1\. How many records (lines) are in this file (look up wc)?

In [152]:
!wc -l advertising_events.csv 

10341 advertising_events.csv


2\. How many unique users are in this file? (hint: consider the 'cut' command and use pipe operator '|')

In [153]:
!cut -f1 -d"," advertising_events.csv | sort |  uniq | wc -l

732


3\. Rank all domains by the number of visits they received in descending order. (hint: consider the 'cut', 'uniq' and 'sort' commands and the pipe operator).

In [315]:
!cut -f 3 -d"," advertising_events.csv | sort |  uniq -c | sort -nr

   3114 google.com
   2092 facebook.com
   1036 youtube.com
   1034 yahoo.com
   1022 baidu.com
    513 wikipedia.org
    511 amazon.com
    382 qq.com
    321 twitter.com
    316 taobao.com


4\. List all records for the user with user id 37. (hint: this can be done using 'grep')

In [182]:
!grep "\<37\>" advertising_events.csv

37,648061658,google.com,0
37,642479972,google.com,2
37,644493341,facebook.com,2
37,654941318,facebook.com,1
37,649979874,baidu.com,1
37,653061949,yahoo.com,1
37,655020469,google.com,3
37,640878012,amazon.com,0
37,659864136,youtube.com,1
37,640361378,yahoo.com,1
37,653862134,facebook.com,0
37,648828970,youtube.com,0


### Part 3: Dealing with data Pythonically (16 Points)

In [2]:
# You might find these packages useful. You may import any others you want!
import pandas as pd
import numpy as np

#1\. Load the data set `ads_dataset.tsv` into a Python Pandas data frame called `ads`.

In [3]:
ads=pd.read_table("ads_dataset.tsv")
ads

,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,Label
0,100000,138.470,51.655,97.827,27.980,0.910,124.711,2.666,3.064,41.928,s
1,100001,160.937,68.768,103.235,48.146,-999.000,-999.000,-999.000,3.473,2.078,b
2,100002,-999.000,162.172,125.953,35.635,-999.000,-999.000,-999.000,3.148,9.336,b
3,100003,143.905,81.417,80.943,0.414,-999.000,-999.000,-999.000,3.310,0.414,b
4,100004,175.864,16.915,134.805,16.405,-999.000,-999.000,-999.000,3.891,16.405,b
...,...,...,...,...,...,...,...,...,...,...,...
1994,101994,99.080,58.041,68.387,9.753,-999.000,-999.000,-999.000,2.771,9.753,b
1995,101995,114.886,27.893,78.988,58.553,-999.000,-999.000,-999.000,2.370,1.901,s
1996,101996,55.789,51.610,43.605,75.560,-999.000,-999.000,-999.000,1.073,2.154,b
1997,101997,208.290,72.995,162.098,78.917,2.486,194.314,-1.265,3.043,4.705,b


2\. Write a Python function called `getDfSummary()` that does the following:
- Takes as input a data frame
- For each variable in the data frame calculates the following features:
  - `number_nan` to count the number of missing not-a-number values
  - Ignoring missing, NA, and Null values:
    - `number_distinct` to count the number of distinct values a variable can take on
    - `mean`, `max`, `min`, `std` (standard deviation), and `25%`, `50%`, `75%` to correspond to the appropriate percentiles
- All of these new features should be loaded in a new data frame. Each row of the data frame should be a variable from the input data frame, and the columns should be the new summary features.
- Returns this new data frame containing all of the summary information

Hint: The pandas `describe()` [(manual page)](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html) method returns a useful series of values that can be used here.

In [316]:
def getDfSummary(input_data):
    output_data=input_data.describe().transpose()
    output_data=output_data.drop('count',axis=1)
    output_data['number_nan']=output_data['number_distinct']=None

    row_index=list(input_data.columns)
    number_nan=input_data.isnull().sum().transpose()
    number_distinct=input_data.nunique()
    output_data['number_nan']=number_nan
    output_data['number_distinct']=number_distinct
    
    return output_data

3\. How long does it take for your `getDfSummary()` function to work on your `ads` data frame? Show us the results below.

Hint: `%timeit getDfSummary(ads)`

In [317]:
# Place your code here
%timeit getDfSummary(ads)

150 ms ± 29.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


4\. Using the results returned from `getDfSummary()`, which fields, if any, contain missing `NaN` values?

In [298]:
# Place your code here
output_ads=getDfSummary(ads)
list(output_ads[output_ads['number_nan']>0].index)

['buy_freq']

5\. For the fields with missing values, does it look like the data is missing at random? Are there any other fields that correlate perfectly, or predict that the data is missing? If missing, what should the data value be?

Hint: create another data frame that has just the records with a missing value. Get a summary of this data frame using `getDfSummary()` and compare the differences. Do some feature distributions change dramatically?

In [314]:
nan_ads=ads[ads.isnull().values==True]
output_nan_ads=getDfSummary(nan_ads)
compare_nan = output_nan_ads.join(output_ads,lsuffix='_nan')
compare_nan

,mean_nan,std_nan,min_nan,25%_nan,50%_nan,75%_nan,max_nan,number_nan_nan,number_distinct_nan,mean,std,min,25%,50%,75%,max,number_nan,number_distinct
isbuyer,0.000000,0.000000,0.0000,0.0,0.0,0.000000,0.00000,0,1,0.042632,0.202027,0.0000,0.0,0.0,0.000000,1.00000,0,2
buy_freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52257,0,1.240653,0.782228,1.0000,1.0,1.0,1.000000,15.00000,52257,10
visit_freq,1.651549,2.147955,1.0000,1.0,1.0,2.000000,84.00000,0,48,1.852777,2.921820,0.0000,1.0,1.0,2.000000,84.00000,0,64
buy_interval,0.000000,0.000000,0.0000,0.0,0.0,0.000000,0.00000,0,1,0.210008,3.922016,0.0000,0.0,0.0,0.000000,174.62500,0,295
sv_interval,5.686388,17.623555,0.0000,0.0,0.0,0.041667,184.91670,0,5112,5.825610,17.595442,0.0000,0.0,0.0,0.104167,184.91670,0,5886
expected_time_buy,0.000000,0.000000,0.0000,0.0,0.0,0.000000,0.00000,0,1,-0.198040,4.997792,-181.9238,0.0,0.0,0.000000,84.28571,0,348
expected_time_visit,-9.669298,31.239030,-187.6156,0.0,0.0,0.000000,91.40192,0,13351,-10.210786,31.879722,-187.6156,0.0,0.0,0.000000,91.40192,0,15135
last_buy,65.741317,53.484622,0.0000,19.0,52.0,106.000000,188.00000,0,189,64.729335,53.476658,0.0000,18.0,51.0,105.000000,188.00000,0,189
last_visit,65.741317,53.484622,0.0000,19.0,52.0,106.000000,188.00000,0,189,64.729335,53.476658,0.0000,18.0,51.0,105.000000,188.00000,0,189
multiple_buy,0.000000,0.000000,0.0000,0.0,0.0,0.000000,0.00000,0,1,0.006357,0.079479,0.0000,0.0,0.0,0.000000,1.00000,0,2


By joining the summary of null data and the summary of ads data, we can see that the data is not missing at random. When buy_freq is missing, it means the customer is not a buyer, that is 'isbuyer' always equals to 0.When custmers do not conduct buying behaviour, correspondingly  'buy_interval' ,'expect_time_buy' and 'multiple_buy' only have 0 value, and their number_distinct equals to 1.

6\. Which variables are binary?

In [300]:
# Place your code here
list(output_ads[output_ads['number_distinct']==2].index)

['isbuyer', 'multiple_buy', 'multiple_visit', 'y_buy']

The binary variables are 'isbuyer', 'multiple_buy', 'multiple_visit' and 'y_buy.'